## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
unique_values = attrition_df.nunique()
print(unique_values)

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64


In [3]:
# Create y_df with the 'Attrition' and 'Department' columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df.head())



  Attrition              Department
0       Yes                   Sales
1        No  Research & Development
2       Yes  Research & Development
3        No  Research & Development
4        No  Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data (excluding 'Attrition' and 'Department')
X_columns = ['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'EducationField',
             'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole']
print(X_columns)


['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole']


In [6]:
# Create X_df using the selected columns
X_df = attrition_df[X_columns]
X_df.head()

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
BusinessTravel,object
DistanceFromHome,int64
Education,int64
EducationField,object
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobRole,object


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Verify the split by checking the shapes of the splits
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (1176, 10)
X_test shape: (294, 10)
y_train shape: (1176, 2)
y_test shape: (294, 2)


In [8]:
# Combine X_train and X_test temporarily to keep columns consistent after encoding
X_combined = pd.concat([X_train, X_test])

# Perform one-hot encoding
X_encoded = pd.get_dummies(X_combined)

# Split them back into training and testing sets
X_train_encoded = X_encoded[:len(X_train)]
X_test_encoded = X_encoded[len(X_train):]

# Check the shapes to confirm
print(f"X_train_encoded shape: {X_train_encoded.shape}")
print(f"X_test_encoded shape: {X_test_encoded.shape}")


X_train_encoded shape: (1176, 25)
X_test_encoded shape: (294, 25)


In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data
X_train_scaled = scaler.fit_transform(X_train_encoded)

# Transform the testing data
X_test_scaled = scaler.transform(X_test_encoded)

# Confirm shapes
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")


X_train_scaled shape: (1176, 25)
X_test_scaled shape: (294, 25)


In [11]:
from sklearn.preprocessing import OneHotEncoder

# Create the encoder
department_encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the training data
y_train_department = department_encoder.fit_transform(y_train[['Department']])

# Transform the testing data
y_test_department = department_encoder.transform(y_test[['Department']])

# Confirm the shapes
print(f"y_train_department shape: {y_train_department.shape}")
print(f"y_test_department shape: {y_test_department.shape}")


y_train_department shape: (1176, 3)
y_test_department shape: (294, 3)


In [12]:
# Create the encoder
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the training data
y_train_attrition = attrition_encoder.fit_transform(y_train[['Attrition']])

# Transform the testing data
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

# Confirm the shapes
print(f"y_train_attrition shape: {y_train_attrition.shape}")
print(f"y_test_attrition shape: {y_test_attrition.shape}")


y_train_attrition shape: (1176, 2)
y_test_attrition shape: (294, 2)


## Part 2: Create, Compile, and Train the Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [14]:
# Get the number of input features
num_columns = X_train_scaled.shape[1]
print(f"Number of input features: {num_columns}")


Number of input features: 25


In [15]:
# Create the input layer
input_layer = layers.Input(shape=(num_columns,))


In [16]:
# First shared layer
shared_layer_1 = layers.Dense(units=64, activation="relu")(input_layer)

# Second shared layer
shared_layer_2 = layers.Dense(units=32, activation="relu")(shared_layer_1)


In [17]:
# Hidden layer for department branch
department_branch = layers.Dense(units=16, activation="relu")(shared_layer_2)

# Output layer for department branch
department_output = layers.Dense(units=3, activation="softmax", name="department_output")(department_branch)


In [18]:
# Hidden layer for attrition branch
attrition_branch = layers.Dense(units=16, activation="relu")(shared_layer_2)

# Output layer for attrition branch
attrition_output = layers.Dense(units=2, activation="softmax", name="attrition_output")(attrition_branch)


In [19]:
# Define the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])


In [20]:
# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})


In [22]:
# Summarize the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,664 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,657 (57.26 KB)

 Trainable params: 4,885 (19.08 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,772 (38.18 KB)

In [23]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_department, 'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - attrition_output_accuracy: 0.8542 - attrition_output_loss: 0.3434 - department_output_accuracy: 0.9894 - department_output_loss: 0.0294 - loss: 0.3729 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.4936 - val_department_output_accuracy: 0.9661 - val_department_output_loss: 0.0659 - val_loss: 0.5804
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8456 - attrition_output_loss: 0.3422 - department_output_accuracy: 0.9907 - department_output_loss: 0.0288 - loss: 0.3707 - val_attrition_output_accuracy: 0.7924 - val_attrition_output_loss: 0.5092 - val_department_output_accuracy: 0.9619 - val_department_output_loss: 0.0846 - val_loss: 0.6227
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8525 - attrition_output_loss: 0.3221 - department_output_accuracy: 0.9925 - department_output_loss: 0.0256 - loss: 0.3477 - val_attrition_output_accuracy: 0.7924 - 

In [25]:
# Evaluate the model with the testing data
loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])

# Print the accuracy for both department and attrition
print(f"Department accuracy: {department_accuracy:.4f}")
print(f"Attrition accuracy: {attrition_accuracy:.4f}")



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - attrition_output_accuracy: 0.7978 - attrition_output_loss: 1.6754 - department_output_accuracy: 0.9639 - department_output_loss: 0.2751 - loss: 1.9750 
Department accuracy: 0.8129
Attrition accuracy: 0.9592


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is a good metric to use, but in this model it mght not be the best because of a class imbalance in the attrition data.
2. I used softmax activation for the department prediction and sigmoid activation for the attrition prediction. Softmax is used for multi-class problems, which was the case in the department prediction. Sigmoid is used for binary classification which was the case in attrition.
3. I think I could do some hyperparameter tuning to change the number of layers, neurons, or rate. I could also try and blanace the class using SMOTE.